In [1]:
import numpy as np

import argparse
import pickle
import string
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score
from collections import Counter
import os
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix 
from tensorflow.contrib import learn
#from tflearn.data_utils import to_categorical, pad_sequences

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scipy import stats
#import tflearn
import json
from models import get_model

In [2]:
def load_data(filename):
    print("Loading data from file: " + filename)
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = [] 
    for i in range(len(data)):
        if(HASH_REMOVE):
            x_text.append(p.tokenize((data[i]['text']).encode('utf-8')))
        else:
            x_text.append(data[i]['text'])
        labels.append(data[i]['label'])
    return x_text,labels

def get_filename(dataset):
    global NUM_CLASSES, HASH_REMOVE
    if(dataset=="twitter"):
        NUM_CLASSES = 3
        HASH_REMOVE = True
        filename = "data/twitter_data.pkl"
    elif(dataset=="formspring"):
        NUM_CLASSES = 2
        filename = "data/formspring_data.pkl"
    elif(dataset=="wiki"):
        NUM_CLASSES = 2
        filename = "data/wiki_data.pkl"
    return filename

In [3]:
def get_embedding_weights(filename, sep):
    embed_dict = {}
    file = open(filename,'r')
    for line in file.readlines():
        row = line.strip().split(sep)
        embed_dict[row[0]] = row[1:]
    print('Loaded from file: ' + str(filename))
    file.close()
    return embed_dict

def map_embedding_weights(embed, vocab, embed_size):
    vocab_size = len(vocab)
    embeddingWeights = np.zeros((vocab_size , embed_size))
    n = 0
    words_missed = []
    for k, v in vocab.iteritems():
        try:
            embeddingWeights[v] = embed[k]
        except:
            n += 1
            words_missed.append(k)
            pass
    print("%d embedding missed"%n, " of " , vocab_size)
    return embeddingWeights

def get_embeddings_dict(vector_type, emb_dim):
    if vector_type == 'sswe':
        emb_dim==50
        sep = '\t'
        vector_file = 'word_vectors/sswe-u.txt'
    elif vector_type =="glove":
        sep = ' '
        if data == "wiki":
            vector_file = 'word_vectors/glove.6B.' + str(emb_dim) + 'd.txt'
        else:
            vector_file = 'word_vectors/glove.twitter.27B.' + str(emb_dim) + 'd.txt'
    else:
        print "ERROR: Please specify a correst model or SSWE cannot be loaded with embed size of: " + str(emb_dim) 
        return None
    
    embed = get_embedding_weights(vector_file, sep)
    return embed

In [4]:
def evaluate_model(model, testX, testY):
    temp = model.predict(testX)
    y_pred  = np.argmax(temp, 1)
    y_true = np.argmax(testY, 1)
    precision = metrics.precision_score(y_true, y_pred, average=None)
    recall = metrics.recall_score(y_true, y_pred, average=None)
    f1_score = metrics.f1_score(y_true, y_pred, average=None)
    print("Precision: " + str(precision) + "\n")
    print("Recall: " + str(recall) + "\n")
    print("f1_score: " + str(f1_score) + "\n")
    print(confusion_matrix(y_true, y_pred))
    print(":: Classification Report")
    print(classification_report(y_true, y_pred))
    return precision, recall, f1_score

In [5]:
def dump_learned_embedding(data, model_type, vector_type, embed_size, embed, vocab_processor):
    vocab = vocab_processor.vocabulary_._mapping
    vocab_size = len(vocab)
    embedDict = {}
    n = 0
    words_missed = []
    for k, v in vocab.iteritems():
        try:
            embeddingDict[v] = embed[k]
        except:
            n += 1
            words_missed.append(k)
            pass
    print("%d embedding missed"%n, " of " , vocab_size)
    
    filename = output_folder_name + data + "_" + model_type + "_" + vector_type + "_" + embed_size + ".pkl"
    with open(filename, 'wb') as handle:
        pickle.dump(embedDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
def get_train_test(data, x_text, labels):
    
    X_train, X_test, Y_train, Y_test = train_test_split( x_text, labels, random_state=42, test_size=0.10)
    
    post_length = np.array([len(x.split(" ")) for x in x_text])
    if(data != "twitter"):
        max_document_length = int(np.percentile(post_length, 95))
    else:
        max_document_length = max(post_length)
    print("Document length : " + str(max_document_length))
    
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length, MAX_FEATURES)
    vocab_processor = vocab_processor.fit(x_text)

    trainX = np.array(list(vocab_processor.transform(X_train)))
    testX = np.array(list(vocab_processor.transform(X_test)))
    
    trainY = np.asarray(Y_train)
    testY = np.asarray(Y_test)
        
    trainX = pad_sequences(trainX, maxlen=max_document_length, value=0.)
    testX = pad_sequences(testX, maxlen=max_document_length, value=0.)

    trainY = to_categorical(trainY, num_classes=NUM_CLASSES)
    testY = to_categorical(testY, num_classes=NUM_CLASSES)
    
    data_dict = {
        "data": data,
        "trainX" : trainX,
        "trainY" : trainY,
        "testX" : testX,
        "testY" : testY,
        "vocab_processor" : vocab_processor
    }
    
    return data_dict

In [7]:
def return_data(data_dict):
    return data_dict["data"], data_dict["trainX"], data_dict["trainY"], data_dict["testX"], data_dict["testY"], data_dict["vocab_processor"]

In [8]:
def shuffle_weights(model, weights=None):
    """Randomly permute the weights in `model`, or the given `weights`.
    This is a fast approximation of re-initializing the weights of a model.
    Assumes weights are distributed independently of the dimensions of the weight tensors
      (i.e., the weights have the same distribution along each dimension).
    :param Model model: Modify the weights of the given model.
    :param list(ndarray) weights: The model's weights will be replaced by a random permutation of these weights.
      If `None`, permute the model's current weights.
    """
    if weights is None:
        weights = model.get_weights()
    weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
    # Faster, but less random: only permutes along the first dimension
    # weights = [np.random.permutation(w) for w in weights]
    model.set_weights(weights)

In [9]:
def train(data_dict, model_type, vector_type, embed_size, dump_embeddings=False):

    data, trainX, trainY, testX, testY, vocab_processor = return_data(data_dict)
    
    vocab_size = len(vocab_processor.vocabulary_)
    print("Vocabulary Size: {:d}".format(vocab_size))
    vocab = vocab_processor.vocabulary_._mapping
    
    print("Running Model: " + model_type + " with word vector initiliazed with " + vector_type + " word vectors.")
    model = get_model(model_type, trainX.shape[1], vocab_size, embed_size, NUM_CLASSES, LEARN_RATE)

    initial_weights = model.get_weights()
    shuffle_weights(model, initial_weights)
    
    if(model_type == 'cnn'):
        if(vector_type!="random"):
            print("Word vectors used: " + vector_type)
            embeddingWeights = tflearn.get_layer_variables_by_name('EmbeddingLayer')[0]
            model.set_weights(embeddingWeights, map_embedding_weights(get_embeddings_dict(vector_type, embed_size), vocab, embed_size))
            model.fit(trainX, trainY, n_epoch = EPOCHS, shuffle=True, show_metric=True, batch_size=BATCH_SIZE)
        else:
            model.fit(trainX, trainY, n_epoch = EPOCHS, shuffle=True, show_metric=True, batch_size=BATCH_SIZE)
    else:
        if(vector_type!="random"):
            print("Word vectors used: " + vector_type)
            model.layers[0].set_weights([map_embedding_weights(get_embeddings_dict(vector_type, embed_size), vocab, embed_size)])
            model.fit(trainX, trainY, epochs=EPOCHS, shuffle=True, batch_size=BATCH_SIZE, 
                  verbose=1)
        else:
            model.fit(trainX, trainY, epochs=EPOCHS, shuffle=True, batch_size=BATCH_SIZE, 
                  verbose=1)
            
    if (dump_embeddings==True):
        if(model_type == 'cnn'):
            embeddingWeights = tflearn.get_layer_variables_by_name('EmbeddingLayer')[0]
        else:
            embed = model.layers[0].get_weights()[0]
    
        embed_filename = output_folder_name + data + "_" + model_type + "_" + vector_type + "_" + str(embed_size) + ".pkl"
        embed.dump(embed_filename)
        
        vocab_filename = output_folder_name + data + "_" + model_type + "_" + vector_type + "_" + str(embed_size) + "_dict.json"
        reverse_vocab_filename = output_folder_name + data + "_" + model_type + "_" + vector_type + "_" + str(embed_size) + "_reversedict.json"
        
        with open(vocab_filename, 'w') as fp:
            json.dump(vocab_processor.vocabulary_._mapping, fp)
        with open(reverse_vocab_filename, 'w') as fp:
            json.dump(vocab_processor.vocabulary_._reverse_mapping, fp)
    
    return  evaluate_model(model, testX, testY)

In [10]:
def print_scores(precision_scores, recall_scores, f1_scores):
    for i in range(NUM_CLASSES):
        print("\nPrecision Class %d (avg): %0.3f (+/- %0.3f)" % (i, precision_scores[:, i].mean(), precision_scores[:, i].std() * 2))
        print( "\nRecall Class %d (avg): %0.3f (+/- %0.3f)" % (i, recall_scores[:, i].mean(), recall_scores[:, i].std() * 2))
        print( "\nF1 score Class %d (avg): %0.3f (+/- %0.3f)" % (i, f1_scores[:, i].mean(), f1_scores[:, i].std() * 2))

In [11]:
def get_data(data, oversampling_rate):
    
    x_text, labels = load_data(get_filename(data)) 
 
    if(data=="twitter"):
        NUM_CLASSES = 3
        dict1 = {'racism':2,'sexism':1,'none':0}
        labels = [dict1[b] for b in labels]
        
        racism = [i for i in range(len(labels)) if labels[i]==2]
        sexism = [i for i in range(len(labels)) if labels[i]==1]
        x_text = x_text + [x_text[x] for x in racism]*(oversampling_rate-1)+ [x_text[x] for x in sexism]*(oversampling_rate-1)
        labels = labels + [2 for i in range(len(racism))]*(oversampling_rate-1) + [1 for i in range(len(sexism))]*(oversampling_rate-1)
    
    else:  
        NUM_CLASSES = 2
        bully = [i for i in range(len(labels)) if labels[i]==1]
        x_text = x_text + [x_text[x] for x in bully]*(oversampling_rate-1)
        labels = list(labels) + [1 for i in range(len(bully))]*(oversampling_rate-1)

    print("Counter after oversampling")
    from collections import Counter
    print(Counter(labels))
    
    filter_data = []
    for text in x_text:
        filter_data.append("".join(l for l in text if l not in string.punctuation))
        
    return x_text, labels

In [12]:
models = [ 'lstm', 'blstm', 'blstm_attention']
word_vectors = ["random", "glove" ,"sswe"]
EPOCHS = 10
BATCH_SIZE = 128
MAX_FEATURES = 2
NUM_CLASSES = None
DROPOUT = 0.25
LEARN_RATE = 0.01
HASH_REMOVE = None
output_folder_name = "results/"

In [13]:
def run_model(data, oversampling_rate, model_type, vector_type, embed_size):    
    x_text, labels = get_data(data, oversampling_rate)
    print(type(x_text), type(labels))
    data_dict = get_train_test(data,  x_text, labels)
    print(type(data_dict))
    precision, recall, f1_score = train(data_dict, model_type, vector_type, embed_size)


In [15]:
data = "formspring"
model_type = "lstm"
vector_type = "random"

for embed_size in [25, 50, 100, 200]:
    run_model(data, 3, model_type, vector_type, embed_size)

Loading data from file: data/formspring_data.pkl
Counter after oversampling
Counter({0: 11997, 1: 2328})
(<type 'list'>, <type 'list'>)
Document length : 62
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
<type 'dict'>
Vocabulary Size: 7190
Running Model: lstm with word vector initiliazed with random word vectors.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 62, 25)            179750    
_________________________________________________________________
dropout (Dropout)            (None, 62, 25)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 25)                5100      
_________________________________________________________

/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
12892/12892 [==============================] - 12s 940us/step - loss: 0.5092 - acc: 0.7851
Epoch 2/10
12892/12892 [==============================] - 10s 809us/step - loss: 0.4613 - acc: 0.8368
Epoch 3/10
12892/12892 [==============================] - 11s 816us/step - loss: 0.4526 - acc: 0.8369
Epoch 4/10
12892/12892 [==============================] - 11s 815us/step - loss: 0.4488 - acc: 0.8370
Epoch 5/10
12892/12892 [==============================] - 10s 810us/step - loss: 0.4422 - acc: 0.8387
Epoch 6/10
12892/12892 [==============================] - 10s 812us/step - loss: 0.3804 - acc: 0.8519
Epoch 7/10
12892/12892 [==============================] - 10s 814us/step - loss: 0.2318 - acc: 0.9140
Epoch 8/10
12892/12892 [==============================] - 10s 814us/step - loss: 0.1559 - acc: 0.9531
Epoch 9/10
12892/12892 [==============================] - 10s 813us/step - loss: 0.1195 - acc: 0.9664
Epoch 10/10
12892/12892 [==============================] - 11s 817us/step - loss: 

Epoch 1/10
12892/12892 [==============================] - 12s 946us/step - loss: 0.4624 - acc: 0.8334
Epoch 2/10
12892/12892 [==============================] - 11s 851us/step - loss: 0.4498 - acc: 0.8369
Epoch 3/10
12892/12892 [==============================] - 11s 851us/step - loss: 0.4444 - acc: 0.8369
Epoch 4/10
12892/12892 [==============================] - 11s 855us/step - loss: 0.4365 - acc: 0.8396
Epoch 5/10
12892/12892 [==============================] - 11s 853us/step - loss: 0.4282 - acc: 0.8459
Epoch 6/10
12892/12892 [==============================] - 11s 853us/step - loss: 0.4244 - acc: 0.8472
Epoch 7/10
12892/12892 [==============================] - 11s 860us/step - loss: 0.4179 - acc: 0.8501
Epoch 8/10
12892/12892 [==============================] - 11s 857us/step - loss: 0.3951 - acc: 0.8503
Epoch 9/10
12892/12892 [==============================] - 11s 852us/step - loss: 0.2386 - acc: 0.9099
Epoch 10/10
12892/12892 [==============================] - 11s 849us/step - loss: 

In [ ]:
data = "formspring"
model_type = "blstm"
vector_type = "random"

for embed_size in [25, 50, 100, 200]:
    run_model(data, 3, model_type, vector_type, embed_size)

Loading data from file: data/formspring_data.pkl
Counter after oversampling
Counter({0: 11997, 1: 2328})
(<type 'list'>, <type 'list'>)
Document length : 62
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
<type 'dict'>
Vocabulary Size: 7190
Running Model: blstm with word vector initiliazed with random word vectors.


/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
12892/12892 [==============================] - 23s 2ms/step - loss: 0.5224 - acc: 0.7683
Epoch 2/10
12892/12892 [==============================] - 21s 2ms/step - loss: 0.3172 - acc: 0.8667
Epoch 3/10
12892/12892 [==============================] - 21s 2ms/step - loss: 0.1591 - acc: 0.9458
Epoch 4/10
12892/12892 [==============================] - 21s 2ms/step - loss: 0.0964 - acc: 0.9670
Epoch 5/10
12892/12892 [==============================] - 21s 2ms/step - loss: 0.0707 - acc: 0.9776: 5s - l
Epoch 6/10
12892/12892 [==============================] - 21s 2ms/step - loss: 0.0529 - acc: 0.9847
Epoch 7/10
12892/12892 [==============================] - 21s 2ms/step - loss: 0.0398 - acc: 0.9881
Epoch 8/10
12892/12892 [==============================] - 21s 2ms/step - loss: 0.0339 - acc: 0.9899
Epoch 9/10
12892/12892 [==============================] - 21s 2ms/step - loss: 0.0272 - acc: 0.9924
Epoch 10/10
12892/12892 [==============================] - 21s 2ms/step - loss: 0.0207 - acc